This notebook is associated with the paper "The relative class number one problem for function fields, I" by K.S. Kedlaya. It runs in SageMath (tested using version 9.6).

In this notebook, we use the linear programming method (a/k/a Weil's explicit formulas) to compute some effective upper bounds on the number of rational points on a curve over a finite field. These bounds are ultimately derived from the positivity condition: for each positive integer $i$, the number of degree-$i$ places of the curve must be nonnegative.

In [1]:
load("../Shared/weil_poly_utils.sage")

## Abelian varieties of order 1 (Section 2)

Verify the minimal polynomials used in the proof of Lemma 2.3.

In [5]:
P.<eta> = QQ[]
K = P.fraction_field()
Q.<x> = K[]
L.<alpha> = K.extension(x^2 + (eta-1)*x -2*eta)
[(alpha^i).minpoly() for i in range(1,5)]

[x^2 + (eta - 1)*x - 2*eta,
 x^2 + (-eta^2 - 2*eta - 1)*x + 4*eta^2,
 x^2 + (eta^3 + 3*eta^2 - 3*eta - 1)*x - 8*eta^3,
 x^2 + (-eta^4 - 4*eta^3 + 2*eta^2 - 4*eta - 1)*x + 16*eta^4]

## Constant extensions (Section 3)

Check the assertion "at most two... do likewise" from the proof of Lemma 3.1.

In [3]:
for (d,n) in [(7,7), (5,30)]:
    K.<eta> = CyclotomicField(n*d)
    Q.<x> = K[]
    pol = x^2 + (eta^d-1)*x -2*eta^d
    for (pol1, _) in pol.factor():
        L.<alpha> = K.extension(pol1)
        ct = 0
        for i in range(d):
            alpha2 = alpha*eta^(i*n)
            eta2 = (alpha2^2 - alpha2)/(2-alpha2)
            if eta2.multiplicative_order() is not Infinity:
                ct += 1
        print(d,n,ct)

7 7 3
7 7 3
5 30 2
5 30 2


## Bounds on rational points on curves (Section 4)

Implement the functions $\psi_d$ associated to a trigonometric polynomial $f(\theta)$ as per equation (5.3.4) in Serre, *Rational Points on Curves over Finite Fields*. Namely, for
$$
f(\theta) = 1 + 2 \sum_{n=1}^\infty c_n \cos n\theta
$$
(input as the list `l` consisting of $c_1, c_2,\dots$) we have
$$
\psi_d(t) = \sum_{n \equiv 0 \pmod{d}} c_n t^n.
$$

In [2]:
RIF = RealIntervalField()

In [3]:
def psi(t, l, n=1):
    P.<z> = LaurentPolynomialRing(RIF)
    c = 1 + 2*sum(i^2 for i in l)
    f = 1/c*(1 + sum(l[i]*(z^(i+1)+z^(-i-1)) for i in range(len(l))))^2
    d = f.degree()
    s = f.coefficients()
    return sum(RIF(t)^i*s[d+i] for i in range(1, d+1) if i%n==0)

Compute constants in a bound of the form
$$
    \#C(\mathbb{F}_q) \leq c_0 g + c_1
$$
where $C$ is a curve of genus $g$ over $\mathbb{F}_q$. We use interval arithmetic to ensure that the bound is rigorous.

In [4]:
l = [RIF(1), RIF(.7), RIF(.2)]
for q in [RIF(2),RIF(3),RIF(4)]:
    print(1/psi(q^-RIF(0.5), l), 1 + psi(q^RIF(0.5), l)/psi(q^-RIF(0.5), l))

0.82595088321150? 5.3452734670778?
1.15251691701742? 11.6683862113728?
1.43462897526502? 21.7491166077739?


Compute constants in a bound of the form
$$
    a_1 + c_2 (2a_2) + c_3 (3a_3) + c_4 (4a_4) \leq c_0 g + c_1
$$
where $a_i$ is the number of degree $i$-places on some curve of genus $g$ over $\mathbb{F}_2$. For convenience, in practice we will round $c_0$ and $c_1$ up and round $c_2,c_3,c_4$ down.

In [5]:
l = [RIF(1), RIF(0.85), RIF(0.25)]
c = [1/psi(1/2^RIF(0.5), l), psi(2^RIF(0.5), l)/psi(1/2^RIF(0.5), l)+1]
print(c)
c = [psi(1/2^RIF(0.5), l, i)/psi(1/2^RIF(0.5), l) for i in range(1,5)]
print(c)

[0.80412436234252?, 5.6189349732825?]
[1.00000000000000?, 0.336683087433209?, 0.138237240714109?, 0.053776916464099?]


## Numerical estimates (Section 5)

In [6]:
q = 3
(11.67 + q)/(q - 1.153)

7.94260963724959

In [7]:
q = 4
(21.75 + q)/(q - 1.435)

10.0389863547758

Table of Weil polynomials of simple abelian varieties of order 1 and dimension at most 6, taken from LMFDB.

In [8]:
P.<T> = QQ[]
simple_poly = {'1.2.ac': T^2 - 2*T + 2,
               '2.2.a_ae': T^4 - 4*T^2 + 4,
               '2.2.ad_f': T^4 - 3*T^3 + 5*T^2 - 6*T + 4,
               '2.2.ac_c': T^4 - 2*T^3 + 2*T^2 - 4*T + 4,
               '2.2.ab_ab': T^4 - T^3 - T^2 - 2*T + 4,
               '3.2.ad_c_b': T^6 - 3*T^5 + 2*T^4 + T^3 + 4*T^2 - 12*T + 8,
               '3.2.ae_j_ap': T^6 - 4*T^5 + 9*T^4 - 15*T^3 + 18*T^2 - 16*T + 8,
               '4.2.af_m_au_bd': T^8 - 5*T^7 + 12*T^6 - 20*T^5 + 29*T^4 - 40*T^3 + 48*T^2 - 40*T + 16,
               '4.2.ae_g_ae_c': T^8 - 4*T^7 + 6*T^6 - 4*T^5 + 2*T^4 - 8*T^3 + 24*T^2 - 32*T + 16,
               '4.2.ad_c_a_b': T^8 - 3*T^7 + 2*T^6 + T^4 + 8*T^2 - 24*T + 16,
               '4.2.ae_f_c_al': T^8 - 4*T^7 + 5*T^6 + 2*T^5 - 11*T^4 + 4*T^3 + 20*T^2 - 32*T + 16,
               '4.2.ae_e_h_av': T^8 - 4*T^7 + 4*T^6 + 7*T^5 - 21*T^4 + 14*T^3 + 16*T^2 - 32*T + 16,
               '4.2.af_n_az_bn': T^8 - 5*T^7 + 13*T^6 - 25*T^5 + 39*T^4 - 50*T^3 + 52*T^2 - 40*T + 16,
               '6.2.ag_p_av_y_abn_cn': T^12 - 6*T^11 + 15*T^10 - 21*T^9 + 24*T^8 - 39*T^7 + 65*T^6 - 78*T^5 + 96*T^4 - 168*T^3 + 240*T^2 - 192*T + 64,
               '6.2.af_j_ah_d_ab_ab': T^12 - 5*T^11 + 9*T^10 - 7*T^9 + 3*T^8 - T^7 - T^6 - 2*T^5 + 12*T^4 - 56*T^3 + 144*T^2 - 160*T + 64,
               '6.2.ag_p_at_g_bb_acj': T^12 - 6*T^11 + 15*T^10 - 19*T^9 + 6*T^8 + 27*T^7 - 61*T^6 + 54*T^5 + 24*T^4 - 152*T^3 + 240*T^2 - 192*T + 64
              }

Compute statistics about simple abelian varieties of order 1, including the *excess* (see Lemma 5.6).

In [9]:
def trace_from_weil_poly(u, n):
    Q.<t> = PowerSeriesRing(QQ)
    v = u.reverse()(t).log()
    l = v.list()
    while len(l) <= n+1:
        l.append(0)
    return [-l[i]*i for i in range(1, n+1)]

In [10]:
cc = [0.8042, 5.619, 1, 0.3366, 0.1137, 0.0537]
min_excess = 5
for u in simple_poly.values():
    d = u.degree() // 2
    tmp = trace_from_weil_poly(u, 4)
    excess = (1 + cc[3])*tmp[0] + cc[3]*tmp[1] + cc[4]*(tmp[2]-tmp[0]) + cc[5]*(tmp[3]-tmp[1])
    excess_scaled = excess / d
    excess_scaled = (excess_scaled*10^4).trunc() * 10.0^(-4)
    min_excess = min(min_excess, excess_scaled)
print(min_excess)

1.56120000000000


Generate LaTeX source for Table 2.

In [11]:
import re

print(r'\begin{tabular}{c||c||c|c|c|c||c|c|c|c}')
print(r'Label & $n$ & $T_{A,2}$ & $T_{A,4}$ & $T_{A,8}$ & $T_{A,16}$ & $T_{A,2} + T_{A,4}$ & $2T_{A,2} + T_{A,4}$ & $3 T_{A,2} + T_{A,4} $ & excess\\')
print(r'\hline')
d0 = 0
for label, u in simple_poly.items():
    tmp = u.factor()
    assert len(tmp) == 1
    F.<a> = NumberField(tmp[0][0])
    eta = (a^2-a)/(2-a)
    n = eta.multiplicative_order()
    assert (n < Infinity)
    tmp = trace_from_weil_poly(u, 4)
    excess = (1 + cc[3])*tmp[0] + cc[3]*tmp[1] + cc[4]*(tmp[2]-tmp[0]) + cc[5]*(tmp[3]-tmp[1])
    excess = (excess*10^4).round() * 10.0^(-4)
    d = u.degree() // 2
    if d0 < d:
        print(r'\hline')
        d0 = d
    excess = excess - d*min_excess
    ans = (n, tmp[0], tmp[1], tmp[2], tmp[3], tmp[0]+tmp[1], 2*tmp[0]+tmp[1], 3*tmp[0]+tmp[1])
    ans = ' & '.join('$' + str(i) + '$' for i in ans)
    ans = r'\avlink{' + re.sub(r'_', r'\_', label) + '} & ' + ans + ' & ' + f'{excess:.4f}'
    print(ans + r' \\')
print(r'\end{tabular}')

\begin{tabular}{c||c||c|c|c|c||c|c|c|c}
Label & $n$ & $T_{A,2}$ & $T_{A,4}$ & $T_{A,8}$ & $T_{A,16}$ & $T_{A,2} + T_{A,4}$ & $2T_{A,2} + T_{A,4}$ & $3 T_{A,2} + T_{A,4} $ & excess\\
\hline
\hline
\avlink{1.2.ac} & $2$ & $2$ & $0$ & $-4$ & $-8$ & $2$ & $4$ & $6$ & 0.0002 \\
\hline
\avlink{2.2.a\_ae} & $1$ & $0$ & $8$ & $0$ & $16$ & $8$ & $8$ & $8$ & 0.0000 \\
\avlink{2.2.ad\_f} & $3$ & $3$ & $-1$ & $0$ & $7$ & $2$ & $5$ & $8$ & 0.6393 \\
\avlink{2.2.ac\_c} & $4$ & $2$ & $0$ & $8$ & $8$ & $2$ & $4$ & $6$ & 0.6626 \\
\avlink{2.2.ab\_ab} & $6$ & $1$ & $3$ & $10$ & $-1$ & $4$ & $5$ & $6$ & 0.0325 \\
\hline
\avlink{3.2.ad\_c\_b} & $7$ & $3$ & $5$ & $6$ & $-11$ & $8$ & $11$ & $14$ & 0.4911 \\
\avlink{3.2.ae\_j\_ap} & $7$ & $4$ & $-2$ & $1$ & $10$ & $2$ & $6$ & $10$ & 0.2929 \\
\hline
\avlink{4.2.af\_m\_au\_bd} & $5$ & $5$ & $1$ & $5$ & $-3$ & $6$ & $11$ & $16$ & 0.5600 \\
\avlink{4.2.ae\_g\_ae\_c} & $8$ & $4$ & $4$ & $4$ & $0$ & $8$ & $12$ & $16$ & 0.2332 \\
\avlink{4.2.ad\_c\_a\_b} & $10$ & 

Compute a lower bound on the excess for $g=8$.

In [12]:
# Values from Lemma 2.3
tr = [[1, -1, 0, 0, 0],
      [1, 2, 1, 0, 0],
      [1, 3, -3, -1, 0],
      [1, 4, -2, 4, 1]]
tr = [vector(v) for v in tr]

In [13]:
v = 1.3366*tr[0] + 0.3366*tr[1] + 0.1137*(tr[2]-tr[0]) + 0.0537*(tr[3]-tr[1]) - vector([1.5612,0,0,0,0])
v

(0.112000000000000, -0.101200000000000, -0.165600000000000, 0.101100000000000, 0.0537000000000000)

In [14]:
tmp = [n for n in range(1, 1000) if euler_phi(n) == 8]
tmp

[15, 16, 20, 24, 30]

In [15]:
for n in tmp:
    trs = [euler_phi(n) / euler_phi(n/gcd(n,d)) * moebius(n/gcd(n,d)) for d in range(1, 5)]
    print(8*v[0] + sum(trs[i-1]*v[i] for i in range(1, 5)))

0.480700000000001
0.896000000000001
0.457400000000001
1.11080000000000
1.08750000000000


Compute a lower bound on the excess for $g \geq 9$.

In [16]:
c = -abs(v[1]) - 2*abs(v[2]) - 3*abs(v[3]) - 4*abs(v[4])
c

-0.950500000000000

In [17]:
9*v[0] + c

0.0575000000000010

Compute upper bounds on $g$ and $g'$ for $q=2$ with no restriction on $d$.

In [18]:
.6732 / 1.5612, 1 + .8042/1.5612, 5.619/1.5612

(0.431206764027671, 1.51511657699206, 3.59915449654112)

In [19]:
def f(g, d):
    ct = 0.6272*g + 9.562
    return 0.4313*ct + (d+2.6) - (d-1.5152)*g

In [20]:
f(41,2), f(40,2)

(-0.0617436399999960, 0.152545000000003)

Improve these bounds using better upper bounds on point counts, taken from Table 1.

In [21]:
f(9,3), f(8,3)

(-1.20450716000000, 0.00978148000000090)

In [22]:
point_count_bounds = {1: 5, 2: 6, 3: 7, 4: 8, 5: 9, 6: 10, 7:10, 8:11}

In [23]:
def f(g, d):
    ct = point_count_bounds[g]
    return 0.4313*ct + (d+2.6) - (d-1.5152)*g

In [24]:
f(8,3), f(7,3), f(6,3)

(-1.53410000000000, -0.480599999999999, 1.00420000000000)

In [25]:
f(6,4), f(5,4), f(4,4)

(-3.99580000000000, -1.94230000000000, 0.111200000000000)

In [26]:
f(4,5), f(3,5)

(-2.88880000000000, 0.164700000000000)

In [27]:
f(3,6), f(2,6)

(-1.83530000000000, 2.21820000000000)